In [ ]:
# Here I load the required packages needed to run the code below

import os
import cv2
import numpy as np
import random
from tqdm import tqdm

In [ ]:
# In this section we specify the folder with the images we want to add noise to. I only select the "train" folder, since there is no need to augment the test or validation sets.
# Next we specify the max amount of noise we want to add to an image. The maximum amount of pixels we to change is 1.85%
# Finally, we initialize the "random" function with a specified seed for reproduceability
TRAIN_DIR = 'tiled_dataset/images/train'   
MAX_NOISE_PERCENTAGE = 0.0185    
SEED = 42
random.seed(SEED)

# In this section we create the "add_salt_and_pepper_noise" which is the main function of the script. The function takes in two arguments, 
# one being which image it is currently augmenting and the second the amount of pixels (in percentage as before) it should change in the image.

def add_salt_and_pepper_noise(image, amount):
    # Makes a copy of the image
    noisy = np.copy(image)
    # Extracts the number of pixels in the image by multiplying the height and width. 
    num_pixels = image.shape[0] * image.shape[1]
    # Calculates the total number of pixels it should change to noise by multiplying the total number of pixels with the percentage of noise (as a fraction)
    num_noisy = int(num_pixels * amount)

    # We then create a for loop which runs for as many loops as the number of pixels the previous section decided we want to change
    for _ in range(num_noisy):
        # Here we select a random pixel by selecting a random height and width value 
        y = random.randint(0, image.shape[0] - 1)
        x = random.randint(0, image.shape[1] - 1)

        # This section functions for both grayscale and colored images, meaning we are able to process images from multiple kinds of datasets.
        # both the if and the else statement do the same however. It looks at the pixel we selected before, and then it uses the random function to provide a random float value
        # between 0 and 1. If the number is below 0.5 the pixel turns white and if it isn't, it turns gray. Note the chance is 50/50. 
        if image.ndim == 2:  # Grayscale
            noisy[y, x] = 0 if random.random() < 0.5 else 255
            print("image was grayscale")
        else:  # Color
            # As mentioned this does the same as before, but here we just have to change all three colour channels to either white or gray
            noisy[y, x] = [0, 0, 0] if random.random() < 0.5 else [255, 255, 255]
    return noisy


# Prints a statement so we know the process has begun
print("Adding salt & pepper noise to training images")
# Loops through all the files in the selected directory, with an added loading bar (tqdm) to show us how far along the process is
for filename in tqdm(os.listdir(TRAIN_DIR)):
    # Here we make sure the program doesn't crash if it runs into a file which isn't an image. Essentially we ask it to skip (continue) if the file doesn't end with .jpg etc.
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    # Creates a path by combining the directory with the current filename
    path = os.path.join(TRAIN_DIR, filename)
    # Reads the current image
    image = cv2.imread(path)

    # Decides the amount of noise to add to the image by randomly selecting a float value (it is a float since MAX_NOISE_PERCENTAGE is a float) 
    # between 0 and 0.0185 (our 1.85% max noise)
    noise_amount = random.uniform(0, MAX_NOISE_PERCENTAGE)

    # Based on this randomly selected amount, it runs the function with the image currently being processed in the for loop and the noise amount just determined
    noisy_image = add_salt_and_pepper_noise(image, noise_amount)

    # Saves the new image in place of the old one in the same folder, as mentioned it is a good idea to backup the dataset for this reason
    cv2.imwrite(path, noisy_image)

# Prints when the process is done
print("Training images updated with salt & pepper noise")